In [2]:
# !python make_dataset.py
# !python train_model.py

In [3]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd
from cascade import utils as cdu

In [4]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [5]:
import os
import sys

sys.path.append(os.path.abspath('..'))
from utils import case

In [6]:
BS = 5

In [7]:
# Overwrite previous run
ModelRepo('repo', overwrite=True)

ModelRepo in repo of 0 lines

In [8]:
train_ds = cdd.Pickler('train_ds').ds()
test_ds = cdd.Pickler('test_ds').ds()
n_features = train_ds.get_meta()[0]['n_features']

model = cdu.SkModel()
model.load('svm')

In [9]:
explainers = {
    'const': ConstantExplainer(n_features=n_features, constant=1),
    'random': RandomExplainer(n_features=n_features, shift=-15, magnitude=10),
    # 'shap': ShapExplainer(train_ds),
    # 'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [10]:
from xaib.cases.feature_importance import CorrectnessCase

from utils import RandomBinBaseline

@case(explainers=explainers, batch_size=BS)
def correctness():
    noisy_model = RandomBinBaseline()

    c = CorrectnessCase(test_ds, model, noisy_model)
    return c

In [11]:
correctness()

100%|██████████| 20/20 [00:00<00:00, 3381.96it/s]


In [12]:
from xaib.cases.feature_importance import ContinuityCase

from utils import NoiseApplier

MULTIPLIER = 0.01

@case(explainers=explainers, batch_size=BS)
def continuity() -> None:
    test_ds_noisy = NoiseApplier(test_ds, multiplier=MULTIPLIER)

    c = ContinuityCase(test_ds, test_ds_noisy, model, multiplier=MULTIPLIER)
    return c

In [13]:
continuity()

20it [00:00, 1434.44it/s]
20it [00:00, 1543.73it/s]


In [14]:
from xaib.cases.feature_importance import ContrastivityCase

@case(explainers=explainers, batch_size=BS)
def contrastivity():
    return ContrastivityCase(test_ds, model)

In [15]:
contrastivity()

100%|██████████| 9/9 [00:00<00:00, 6922.56it/s]


In [16]:
from xaib.cases.feature_importance import CoherenceCase

@case(explainers=explainers, expls=list(explainers.values()), batch_size=BS)
def coherence():
    return CoherenceCase(test_ds, model)

In [17]:
coherence()

100%|██████████| 20/20 [00:00<00:00, 1774.73it/s]


In [18]:
from xaib.cases.feature_importance import CompactnessCase

@case(explainers=explainers, batch_size=BS)
def compactness():
    return CompactnessCase(test_ds, model)

In [19]:
compactness()

100%|██████████| 20/20 [00:00<00:00, 487.52it/s]


In [20]:
from xaib.cases.feature_importance import CovariateComplexityCase

@case(explainers=explainers, batch_size=BS)
def covariate_complexity():
    return CovariateComplexityCase(test_ds, model)

In [21]:
covariate_complexity()

100%|██████████| 20/20 [00:00<00:00, 10009.08it/s]


In [22]:
repo = ModelRepo('repo')

In [23]:
t = MetricViewer(repo).table

In [24]:
t

,line,num,created_at,saved,parameter_randomization_check,name,small_noise_check,multiplier,label_difference,other_disagreement,sparsity,covariate_regularity
0,repo\00000,0,2023-02-25 19:39:48.784778+00:00,a few seconds after,0.00000,const,NaN,NaN,NaN,NaN,NaN,NaN
1,repo\00000,1,2023-02-25 19:39:48.784778+00:00,a few seconds after,1.28651,random,NaN,NaN,NaN,NaN,NaN,NaN
2,repo\00001,0,2023-02-25 19:39:52.750110+00:00,a few seconds after,NaN,const,0.000000,0.01,NaN,NaN,NaN,NaN
3,repo\00001,1,2023-02-25 19:39:52.750110+00:00,a few seconds after,NaN,random,1.276685,0.01,NaN,NaN,NaN,NaN
4,repo\00002,0,2023-02-25 19:39:54.024788+00:00,a few seconds after,NaN,const,NaN,NaN,0.000000,NaN,NaN,NaN
5,repo\00002,1,2023-02-25 19:39:54.024788+00:00,a few seconds after,NaN,random,NaN,NaN,1.360522,NaN,NaN,NaN
6,repo\00003,0,2023-02-25 19:39:55.267382+00:00,a few seconds after,NaN,const,NaN,NaN,NaN,0.910392,NaN,NaN
7,repo\00003,1,2023-02-25 19:39:55.267382+00:00,a few seconds after,NaN,random,NaN,NaN,NaN,1.579110,NaN,NaN
8,repo\00004,0,2023-02-25 19:39:56.752295+00:00,a few seconds after,NaN,const,NaN,NaN,NaN,NaN,0.000000,NaN
9,repo\00004,1,2023-02-25 19:39:56.752295+00:00,a few seconds after,NaN,random,NaN,NaN,NaN,NaN,0.326426,NaN
